In [1]:
%%capture
%matplotlib inline

from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

# Initialization cell
import os
import subprocess

import pandas as pd
import numpy as np

path_to_NPDtools = '/home/letovesnoi/work/tools/NPDtools-2.4.0-Linux/'

data_dir = 'data'
gnps_library = os.path.join(data_dir, 'GNPS-LIBRARY.mgf')
# gnps_library = os.path.join(data_dir, 'tst.mgf')

spectra_dir = os.path.join(data_dir, 'spectra')
mols_dir = os.path.join(data_dir, 'mols')
fthr = os.path.join(data_dir, 'data.fthr')

reg_run_dir = os.path.join(data_dir, 'REG_RUN_GNPS')
fdr0_tsv = os.path.join(reg_run_dir, 'regrun_fdr0_complete.tsv')
spectra_rr_dir = os.path.join(data_dir, 'spectra_REG_RUN')
reg_run_df = pd.read_csv(fdr0_tsv, sep="\t")
grouped_df = reg_run_df.groupby(['SpecFile'])['LocalSpecIdx', 'Structure'].agg(list)
fthr_rr = os.path.join(data_dir, 'data_rr.fthr')

discrete_masses = np.linspace(0, 5000, num=50000, dtype='float64')

In [2]:
# create directory for MGF files
if os.path.exists(spectra_dir):
    subprocess.call('rm -r {}'.format(spectra_dir), shell=True)
os.mkdir(spectra_dir)

# create directory for Molfiles
if os.path.exists(mols_dir):
    subprocess.call('rm -r {}'.format(mols_dir), shell=True)
os.mkdir(mols_dir)

In [3]:
def get_header(fin):
    header = []
    line = fin.readline()
    while '=' in line:
        ind = line.find('=')
        header.append((line[:ind], line[ind + 1:].strip()))
        line = fin.readline()
    return header, line

def get_spectrum(fin):
    header, line = get_header(fin)
    intensity = []
    while line != 'END IONS\n':
        intensity.append(line.strip().split())
        line = fin.readline()
    return header, np.array(intensity, dtype='float64')

def get_spectrum_id(header):
    return header[18][1]

def write_spectrum(header, intensity, spectrum_path):
    with open(spectrum_path, 'a') as fout:
        fout.write('BEGIN IONS\n')
        for key, value in header:
            fout.write(key + '=' + value + '\n')
        for mass, abundance in intensity:
            fout.write(str(mass) + ' ' + str(abundance) + '\n')
        fout.write('END IONS\n')

def split_mgf(mgf_library, spectra_dir, local_ids=None):
    spectra_pathes = []
    with open(mgf_library, 'r') as fin:
        num_spectra = 0
        while True:
            line = fin.readline()
            if not line:
                break
            header, intensity = get_spectrum(fin)
            if not local_ids:
                spectrum_id = get_spectrum_id(header)
            elif num_spectra in local_ids:
                mgf_name = os.path.splitext(os.path.basename(mgf_library))[0]
                spectrum_id = mgf_name + '_' + str(num_spectra)
            else:
                continue
            spectra_pathes.append(os.path.join(spectra_dir, spectrum_id + '.mgf'))
            write_spectrum(header, intensity, spectra_pathes[-1])
            num_spectra += 1
    # print('{} spectra'.format(num_spectra))
    return spectra_pathes

In [ ]:
peptidic = 0
smiles_errors = 0
known = 0
empty = 0

def get_smiles(header):
    return header[11][1]

def get_mol(spectrum_path):
    with open(spectrum_path, 'r') as fin:
        fin.readline()
        header, intensity = get_spectrum(fin)
        molfile = os.path.join(mols_dir, get_spectrum_id(header) + '.mol')
        # print('SMILES=' + get_smiles(header))
        # print('SPECTRUMID=' + get_spectrum_id(header))
        subprocess.call('molconvert mol:V3+H -s \'{}\' -o {}'.format(get_smiles(header), molfile), shell=True)
    return molfile

def is_structural(mgf_file):
    global known
    with open(mgf_file, 'r') as fin:
        fin.readline()
        header, intensity = get_spectrum(fin)
        if get_smiles(header) == 'N/A' or get_smiles(header) == '':
            return False
        else:
            known += 1
            return True

def is_peptidic(molfile):
    global smiles_errors, peptidic
    tmp_peptidic = os.path.join(data_dir, 'peptidic.tmp')
    subprocess.call('{0}bin/print_structure {1} -C {0}share/npdtools/ --print_rule_fragmented_graph > {2}'.format(path_to_NPDtools, molfile, tmp_peptidic), shell=True)
    with open(tmp_peptidic, 'r') as fin:
        line = fin.readline()
        subprocess.call('rm {}'.format(tmp_peptidic), shell=True)
        if 'number of components' in line:
            num_components = line.strip().split(':')[1][1:]
        else:
            smiles_errors += 1
            print('{} contains an error'.format(molfile))
            return -1
    if int(num_components) > 3:
        peptidic += 1
        return 1
    else:
        return 0

def is_empty(mgf_file):
    global empty
    with open(mgf_file, 'r') as fin:
        fin.readline()
        header, intensity = get_spectrum(fin)
        if len(intensity) != 0:
            return False
        else:
            empty += 1
            print('{} has no information about intensity'.format(mgf_file))
            return True
        
def filter_spectra():
    for mgf_file in tqdm(split_mgf(gnps_library, spectra_dir)):
        # remove unknown spectrum
        if not is_structural(mgf_file) or is_empty(mgf_file):
            subprocess.call('rm {}'.format(mgf_file), shell=True)
        else:
            molfile = get_mol(mgf_file)
            # remove non-peptidic data
            is_pep = is_peptidic(molfile)
            if is_pep == 0 or is_pep == -1:
                subprocess.call('rm {}'.format(mgf_file), shell=True)
                subprocess.call('rm {}'.format(molfile), shell=True)
                
filter_spectra()

print('{} have a known structure'.format(known))
print('{} have no information about intensity'.format(empty))
print('{} SMILES contains an error'.format(smiles_errors))
print('{} peptidic among structural, with non zero intensity, and with correct SMILES'.format(peptidic))

data/spectra/CCMSLIB00000078897.mgf has no information about intensity


In [ ]:
def get_fthr(spectra_pathes, fthr, discrete_masses):
    spectra_df = pd.DataFrame()
    for mgf_file in tqdm(spectra_pathes):
        with open(mgf_file, 'r') as fin:
            fin.readline()
            header, intensity = get_spectrum(fin)
        id = os.path.splitext(os.path.basename(mgf_file))[0]
        bins = pd.cut(intensity[:, 0], bins=discrete_masses, labels=False)
        spectrum_df = pd.DataFrame({id: intensity[:, 1], 'binned': bins}).groupby(['binned']).sum().T
        spectra_df = pd.concat([spectra_df, spectrum_df], sort=True)
    spectra_df.columns = spectra_df.columns.astype(str)
    spectra_df.reset_index().to_feather(fthr)
    return fthr

spectra_pathes = [os.path.join(spectra_dir, sp) for sp in os.listdir(spectra_dir)]

fthr = get_fthr(spectra_pathes, fthr, discrete_masses)

In [ ]:
# create directory for MGF files
if os.path.exists(spectra_rr_dir):
    subprocess.call('rm -r {}'.format(spectra_rr_dir), shell=True)
os.mkdir(spectra_rr_dir)

spectra_rr_pathes = []
for index, row in grouped_df.iterrows():
    mgf_path = os.path.join(reg_run_dir, os.path.splitext(index)[0] + '_filtered.mgf')
    spectra_rr_pathes.extend(split_mgf(mgf_path, spectra_rr_dir, row['LocalSpecIdx']))
    
fthr_rr = get_fthr(spectra_rr_pathes, fthr_rr, discrete_masses)

In [ ]:
# Skip other cells except initialization and run this
def get_df_from_files(fthr,  fthr_rr):
    spectra_df = pd.read_feather(fthr)
    del spectra_df['index']
    spectra_df.columns = pd.to_numeric(spectra_df.columns)

    spectra_rr_df = pd.read_feather(fthr_rr)
    del spectra_rr_df['index']
    spectra_rr_df.columns = pd.to_numeric(spectra_rr_df.columns)

    df = pd.concat([spectra_df, spectra_rr_df], sort=True)
    
    df = df.fillna(0)
    
    display(df)
    
    return df

df = get_df_from_files(fthr,  fthr_rr)

In [ ]:
# Skip it!  
import matplotlib.pyplot as plt

# Test discretisation
def plot_discretisation(tst_d_m=discrete_masses):
    i = 0
    fig = plt.figure(figsize=(20, 8))
    for mgf_file in tqdm(spectra_pathes):
        if i == 32:
            break
        ax = fig.add_subplot(4, 8, i + 1, xticks=[], yticks=[])
        with open(mgf_file, 'r') as fin:
            fin.readline()
            header, intensity = get_spectrum(fin)
        id = os.path.splitext(os.path.basename(mgf_file))[0]
        ax.plot(intensity[:, 0], intensity[:, 1])
        bins = pd.cut(intensity[:, 0], bins=tst_d_m, labels=False)
        tst_df = pd.DataFrame({'intensity': intensity[:, 1], 'binned': bins}).groupby(['binned'], as_index=False).sum()
        tst_df['x'] = tst_df['binned'].transform(lambda b: tst_d_m[b])
        ax.plot(tst_df.x, tst_df.intensity)
        i += 1
    return ax
        
plt.show(plot_discretisation())

# Try different discretizations:
discr_list = [100, 500, 1000, 5000, 10000]
fthr_list = []
fthr_rr_list = []
df_list = []
for discr_n in discr_list:
    print('Discretization: {}\n'.format(discr_n))
    discr_m = np.linspace(0, 5000, num=discr_n, dtype='float64')
    
    fthr_d = os.path.join(data_dir, 'data.' + str(discr_n) + '.fthr')
    fthr_list.append(get_fthr(spectra_pathes, fthr_d, discr_m))
    
    fthr_rr_d = os.path.join(data_dir, 'data_rr.' + str(discr_n) + '.fthr')
    fthr_rr_list.append(get_fthr(spectra_rr_pathes, fthr_rr_d, discr_m))
    
    df_list.append(get_df_from_files(fthr_d,  fthr_rr_d))
    
    plt.show(plot_discretisation(discr_m))

In [ ]:
# Get labels
def get_spectrum_cyclicality(molfile):
    tmp_cyclicality = os.path.join(data_dir, 'cyclicality.tmp')
    subprocess.call('{0}bin/print_structure {1} -C {0}share/npdtools/ --print_structure > {2}'.format(path_to_NPDtools, molfile, tmp_cyclicality), shell=True)
    with open(tmp_cyclicality, 'r') as fin:
        cyclicality = fin.readline().strip()
    subprocess.call('rm {}'.format(tmp_cyclicality), shell=True)
    return cyclicality

cycl_dict = {'linear': 0, 'cyclic': 1, 'b-cyclic': 2, 'branch-cyclic': 2, 'complex': 3}

def get_labels(mols_dir):
    labels = []
    for molfile in tqdm(os.listdir(mols_dir)):
        cyclicality = get_spectrum_cyclicality(os.path.join(mols_dir, molfile))
        labels.append(cycl_dict[cyclicality])
    return labels    

def get_labels_rr(grouped_df):
    labels = []
    for index, row in grouped_df.iterrows():
        labels.extend([cycl_dict[cyclicality] for cyclicality in row['Structure']])
    return labels

labels = get_labels(mols_dir) + get_labels_rr(grouped_df)

print('{} linear'.format(labels.count(0)))
print('{} cyclic'.format(labels.count(1)))
print('{} branch-cyclic'.format(labels.count(2)))
print('{} complex'.format(labels.count(3)))

In [ ]:
# Remove complex and branch-cyclic classes
nl = np.array(labels)
criteria = np.logical_or(nl == 0, nl == 1) 
l_2 = nl[criteria]
df_list_2 = [dataframe[criteria] for dataframe in df_list]
df_2 = df[criteria]

In [ ]:
import keras
from sklearn.model_selection import train_test_split

choosen_df = df_list_2[0]

X = choosen_df.to_numpy().astype('float32')
y = np.array(l_2).astype('float32')

display(choosen_df)

X_train, X_test, train_binary_labels, test_binary_labels = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_3d = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_3d = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# one-hot encode the labels
num_classes = len(np.unique(train_binary_labels))
y_train = keras.utils.to_categorical(train_binary_labels, num_classes)
y_test = keras.utils.to_categorical(test_binary_labels, num_classes)

print(X_train_3d.shape)

In [ ]:
# Benchmark model
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy='most_frequent', random_state=42).fit(X_train, y_train)

In [ ]:
# Try clustering
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import precision_score

for dataframe in df_list_2:
    kmeans_pred = KMeans(n_clusters=2, random_state=42).fit_predict(X)
    unique, counts = np.unique(kmeans_pred, return_counts=True)
    print('Kmeans predictions:', dict(zip(unique, counts)))

#     gmm_pred = GaussianMixture(n_components=2, covariance_type='full', random_state=42).fit_predict(X)
#     unique, counts = np.unique(gmm_pred, return_counts=True)
#     print('GMM predictions:', dict(zip(unique, counts)))

In [ ]:
# Try SVM
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# C, kernel, degree, gamma
# parameters = {'kernel': ['poly', 'rbf'], 'gamma': [4, 16, 32], 'degree': [2, 4]}
# svc = SVC(class_weight='balanced')
# svc_clf = GridSearchCV(svc, parameters, cv=5, scoring=make_scorer('roc_auc'))
svc_clf = SVC(class_weight='balanced')
svc_clf.fit(X_train, train_binary_labels)
# print(svc_clf.best_estimator_)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

cnn_clf = Sequential()
# model.add(Conv2D(filters=4, kernel_size=4, padding='same', activation='relu', input_shape=(1, 23325, 1)))
# model.add(MaxPooling2D(pool_size=(1, 2)))
# model.add(Conv2D(filters=4, kernel_size=4, padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(1,2)))
cnn_clf.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu', 
                   input_shape=(X_train_3d.shape[1], 1)))
cnn_clf.add(MaxPooling1D(pool_size=2))
cnn_clf.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
cnn_clf.add(MaxPooling1D(pool_size=2))
cnn_clf.add(Dropout(0.3))
cnn_clf.add(Flatten())
cnn_clf.add(Dense(64, activation='relu'))
cnn_clf.add(Dropout(0.4))
cnn_clf.add(Dense(num_classes, activation='softmax'))
cnn_clf.summary()

In [ ]:
# compile the model
cnn_clf.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
# train the model
from keras.callbacks import ModelCheckpoint
from sklearn.utils import class_weight

checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, save_best_only=True)
w = dict(enumerate(class_weight.compute_class_weight('balanced', np.unique(train_binary_labels), train_binary_labels)))
print('class weight = ', w, '\n')
hist = cnn_clf.fit(X_train_3d, y_train, batch_size=32, epochs=25, validation_split=0.2, 
                 callbacks=[checkpointer], verbose=2, shuffle=True, class_weight=w)

In [ ]:
# load the weights that yielded the best validation accuracy
cnn_clf.load_weights('model.weights.best.hdf5')

In [ ]:
# Evaluation
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score

def evaluate_imbalanced(clf):
    type_clf = type(clf).__name__
    if type_clf == 'Sequential':
        x = X_test_3d
    else:
        x = X_test
    y_pred = clf.predict(x)
    if type_clf != 'SVC':
        pred_binary_labels = np.argmax(y_pred, axis=1)
        y_score = y_pred[:, 1]
    else:
        pred_binary_labels = y_pred
        y_score = y_pred
    tn, fp, fn, tp = confusion_matrix(test_binary_labels, pred_binary_labels).ravel()
    print('{name} TN = {tn}, {name} FP = {fp}, {name} FN = {fn}, {name} TP = {tp}'.
          format(tn=tn, fp=fp, fn=fn, tp=tp, name=type_clf))
    fpr, tpr, thresholds = roc_curve(test_binary_labels, y_score)
    auc_score = auc(fpr, tpr)
    return fpr, tpr, auc_score


fpr_d, tpr_d, auc_d = evaluate_imbalanced(dummy_clf)
fpr_cnn, tpr_cnn, auc_cnn = evaluate_imbalanced(cnn_clf)
fpr_svc, tpr_svc, auc_svc = evaluate_imbalanced(svc_clf)

# Plot ROC curves for these two models
plt.figure()
plt.plot(fpr_d, tpr_d, lw=2, label='Dummy ROC curve (area = %0.2f)' % auc_d)
plt.plot(fpr_cnn, tpr_cnn, lw=2, label='CNN ROC curve (area = %0.2f)' % auc_cnn)
plt.plot(fpr_svc, tpr_svc, lw=2, label='SVM ROC curve (area = %0.2f)' % auc_svc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Visualize some predictions
import matplotlib.pyplot as plt

# define text labels
names = ['linear', 'cyclic', 'b-cyclic', 'complex']

# plot a random sample of test images, their predicted labels, and ground truth
fig = plt.figure(figsize=(20, 8))
y_cnn = cnn_clf.predict(X_test_3d)
for i, rc_i in enumerate(np.random.choice(X_test.shape[0], size=32, replace=False)):
    ax = fig.add_subplot(4, 8, i + 1, xticks=[], yticks=[])    
    x = np.array([discrete_masses[int(bin_i)] for bin_i in choosen_df.columns.values.tolist()])
    y = np.array(X_test[rc_i])  
    y_mask = (y != 0)
    ax.plot(x[y_mask], y[y_mask])
    pred_idx = np.argmax(y_cnn[rc_i])
    true_idx = int(test_binary_labels[rc_i])
    ax.set_title("{} ({})".format(names[pred_idx], names[true_idx]), 
                 color=("green" if pred_idx == true_idx else "red"))